In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [3]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [6]:
# reflect an existing database into a new model
Base = automap_base()

In [7]:
# reflect the tables
Base.prepare(engine,reflect=True)

In [8]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [9]:
# Save references to each table Station
Station = Base.classes.station

In [ ]:
# Save references to each table Measurement
Measurement = Base.classes.measurement

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
most_recent_date = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
most_recent_date

In [ ]:
# Calculate the date one year from the last date in data set.
one_year_from_last_date = dt.date(2017,8,23) - dt.timedelta(days=365)
one_year_from_last_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Need to filter for just last 12 months, as data set has items going back to Jan 1, 2010
precipitation_date = session.query(Measurement.date,Measurement.prcp).\
    filter(Measurement.date >=one_year_from_last_date).\
    order_by(Measurement.date).all()
precipitation_date

In [ ]:
# Perform a query to retrieve the data and precipitation scores
# Need to call Cols B & C in hawaii_measurements.csv

data_and_precipitation_score = session.query(Measurement.date,Measurement.prcp).order_by(Measurement.date.desc()).all()
data_and_precipitation_score

In [ ]:
# Save the query results as a Pandas DataFrame and set the index to the date column
# Need to reference precipitation_date
precipitation_dataframe = pd.DataFrame(precipitation_date, columns = ["date", "Precipitation"])


In [ ]:
# Sort the dataframe by date
precipitation_dataframe.set_index("date", inplace=True,)
precipitation_dataframe


In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
precipitation_dataframe.plot(title="Exploratory Precipitation Analysis",figsize=(20,10))
plt.legend(loc="upper right")
plt.show()
plt.savefig("Exploratory Precipitation Analysis.png")

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipitation_dataframe.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
# Unique stations 
total_number_stations = session.query(Measurement.station).distinct().count()
total_number_stations

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
most_active_stations = session.query(Measurement.station, func.count(Measurement.station)).\
    group_by(Measurement.station).\
    order_by(func.count(Measurement.station).desc()).all()
    

In [ ]:
# List the stations and the counts in descending order.
most_active_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest temperature.
sel = [func.min(Measurement.tobs)]
lowest_temperature = session.query(*sel).\
    filter(Measurement.station == "USC00519281").all()
lowest_temperature

In [ ]:
# Using the most active station id from the previous query, calculate the highest temperature.
sel = [func.max(Measurement.tobs)]
highest_temperature = session.query(*sel).\
    filter(Measurement.station == "USC00519281").all()
highest_temperature

In [ ]:
# Using the most active station id from the previous query, calculate the average temperature.
sel = [func.avg(Measurement.tobs)]
average_temperature = session.query(*sel).\
    filter(Measurement.station == "USC00519281").all()
average_temperature

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station 
# Station USC00519281

temperature_observation_data = session.query(Measurement.tobs).\
    filter(Measurement.date >=one_year_from_last_date).\
    order_by(Measurement.date).all()

In [ ]:
# Use Pandas Plotting with Matplotlib to plot the data
temperature_observation_data = pd.DataFrame(temperature_observation_data, columns = ["TOBS"])

In [ ]:
# Plot the results as a histogram
temperature_observation_data.plot.hist(bins=12, title= "Temperature vs Frequency", figsize = (15,10))
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.legend(loc = "upper right")
plt.show()
plt.savefig("Temperature vs Frequency.png")

# Close session

In [ ]:
# Close Session
session.close()